# Tutorial 9: Automatic Task Generation with Zero-Learn

One of the most significant bottlenecks in training a new agent is creating a high-quality, diverse training dataset. Manually writing dozens or hundreds of example queries is tedious and may not cover all the edge cases your agent needs to learn.

ToolBrain's "zero-learn" capability, exposed through the `brain.generate_training_examples()` method, solves this problem by using a language model to automatically generate training data for you.

This tutorial is based on the example in `examples/03_generate_training_examples.py`.

## The Problem: The Cold Start

When you've just created a new agent with a new set of tools, you have a "cold start" problem: you have no data to train it on. How do you teach a finance agent to use a `calculate_compound_interest` tool without manually writing every possible question a user might ask about it?

## The Solution: `generate_training_examples()`

This method leverages a language model to act as a creative user, generating a variety of queries that are relevant to the tools you've provided. You simply give it a high-level description of the task, and it does the rest.

**How it works:**

1.  You provide a `task_description` that outlines the agent's purpose (e.g., "a finance agent that can calculate interest and loan payments").
2.  You also provide the agent's tools.
3.  The method prompts an LLM, asking it to generate a specified number of user queries that would require the use of these tools to answer.
4.  Optionally, it can even use the same LLM to rank the generated examples, ensuring the most relevant and well-formed queries are prioritized.

## The Code: `03_generate_training_examples.py`

Let's see how this works in the finance agent example.

### Step 1: Define Tools and an Agent

First, we define a set of tools for a finance agent, such as `calculate_compound_interest`, `calculate_loan_payment`, etc. Then we create an agent with these tools.

In [1]:
from smolagents import tool
from toolbrain import Brain, create_agent

/home/quan/MyProjects/ToolBrain/ToolBrain/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/quan/MyProjects/ToolBrain/ToolBrain/.venv/lib/python3.12/site-packages/toolbrain/adapters.py:9: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
@tool
def calculate_compound_interest(principal: float,
                                rate: float,
                                times_compounded: int,
                                years: float) -> float:
    """
    Calculate compound interest.

    Args:
        principal (float): The principal amount.
        rate (float): Annual interest rate (as a decimal, e.g., 0.05 for 5%).
        times_compounded (int): Number of times interest is compounded per year.
        years (float): Number of years.

    Returns:
        float: Final amount after compounding.
    """
    return principal * (1 + rate / times_compounded) ** (times_compounded * years)


@tool
def calculate_loan_payment(principal: float,
                           annual_rate: float,
                           years: int) -> float:
    """
    Calculate monthly loan payment using the annuity formula.

    Args:
        principal (float): Loan amount.
        annual_rate (float): Annual interest rate (decimal, e.g., 0.06).
        years (int): Loan term in years.

    Returns:
        float: Monthly payment amount.
    """
    r = annual_rate / 12
    n = years * 12
    return principal * (r * (1 + r)**n) / ((1 + r)**n - 1)


@tool
def calculate_roi(gain: float, cost: float) -> float:
    """
    Calculate return on investment (ROI).

    Args:
        gain (float): Total gain from investment.
        cost (float): Total cost of investment.

    Returns:
        float: ROI as a decimal.
    """
    return (gain - cost) / cost


@tool
def calculate_cagr(begin_value: float,
                   end_value: float,
                   years: float) -> float:
    """
    Calculate Compound Annual Growth Rate (CAGR).

    Args:
        begin_value (float): Initial value.
        end_value (float): Final value.
        years (float): Number of years.

    Returns:
        float: CAGR as a decimal.
    """
    return (end_value / begin_value) ** (1 / years) - 1


@tool
def calculate_npv(rate: float,
                  cash_flows: list[float]) -> float:
    """
    Calculate Net Present Value (NPV).

    Args:
        rate (float): Discount rate (as a decimal).
        cash_flows (list[float]): List of cash flows per period (negative for outflows, positive for inflows).

    Returns:
        float: Net present value.
    """
    return sum(cf / (1 + rate) ** t for t, cf in enumerate(cash_flows))

### Step 2: Define the Task

We write a high-level description of the kinds of tasks we want to generate. This guides the LLM in its generation process.


In [6]:
description = (
    "Generate task to learn to use simple finance tools such as calculating interest, converting currencies, or computing profit. "
    "The prompts should include varied numeric inputs, realistic edge cases (zero values, high rates), and require validation of outputs."
)
# there is a helper function to create agent if you prefer to
# Create an agent with a suite of finance tools
agent = create_agent(
    model_id="Qwen/Qwen2.5-0.5B-Instruct",
    tools=[
        calculate_compound_interest,
        calculate_loan_payment,
        calculate_cagr,
        calculate_npv
    ],
)

==((====))==  Unsloth 2025.9.2: Fast Qwen2 patching. Transformers: 4.55.3.
   \\   /|    NVIDIA RTX 6000 Ada Generation. Num GPUs = 1. Max memory: 47.5 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


### Step 3: Generate the Examples

Now, we call `brain.generate_training_examples()`, passing in our description and specifying how many examples we want.


In [8]:
brain = Brain(agent=agent)
generated_examples = brain.generate_training_examples(
    task_description=description,
    num_examples=20,
    min_tool_calls=2,
    max_words=80,
    external_model=None,
    external_tools=None,
    guidance_example=None,
    self_rank=True
)

The method will return a list of strings, where each string is a synthetically generated user query.
We print out the generated examples

In [9]:
print("Generated Training Examples:")
for i, example in enumerate(generated_examples, 1):
    print(f"{i}. {example}\n")

Generated Training Examples:
1. [Gold answer] I'm ready to help with various financial calculations! Please provide me with the details needed to proceed.

2. [Gold answer]

3. [Gold answer] How would you calculate your net worth over time?

4. [Gold answer] I am ready to help! Please provide me with the details needed for each finance-related task.

5. [Gold answer] Please refer to the following financial calculators for assistance:

1. Calculate Interest - This tool calculates the interest earned on a principal amount over a specified number of years at a given annual interest rate.
   Args: {'principal': {'type': 'number', 'description': 'Principal amount.'}, 'rate': {'type': 'number', 'description': 'Annual interest rate.'}, 'time_years': {'type': 'number', 'description': 'Time in years.'}}
   Returns: float: Interest earned.

2. Calculate Loan Payment - This tool calculates the monthly loan payment based on the annuity formula.
   Args: {'principal': {'type': 'number', 'descriptio

### Step 4: Use the Generated Data for Training

This generated list of queries can be immediately converted into a training dataset and used to train your agent. This allows you to go from a new set of tools to a trained agent in a single script, with no manual data creation.


In [ ]:
# Use generated examples to train the agent
# You can also manually curate or modify the generated examples before training.
training_dataset = [{"query": example} for example in generated_examples]
brain.train(training_dataset)


🚀 Starting training...

--- Iteration 1/1 ---

🔄 Training step for query: '[Gold answer] I'm ready to help with various finan...'
Calling get_trace with query: '[Gold answer] I'm ready to help with various finan...'
Enable tool retrieval: False
    📝 Trace 1/2


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ [Gold answer\] I'm ready to help with various financial calculations! Please provide me with the details needed │
│ to proceed.                                                                                                     │
│                                                                                                                 │
╰─ UnslothModel - Qwen/Qwen2.5-0.5B-Instruct ─────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in code parsing:
Your code snippet is invalid, because the regex pattern <code>(.*?)</code> was not found in it.
            Here is your code snippet:
            Sure, let's start by determining the current age of the Pope based on Wikipedia information:

1. We know the Pope is currently 88 years old.
2. The website provides this calculation: "The pope Francis is currently 88 years old."
3. This calculation works by comparing the current date (2023) to the Pope's birthdate (15th July 1952). 

Let's perform the calculation using the correct formula:

```
age = 2023 - 1573
```

This gives us:

```
age = 2023 - 1573
```

We can verify the result:

```
2023 - 1573 = 450
```

Therefore, the current age of the Pope, rounded to the nearest whole number, is 450 years old.

If you would like to continue with the task of calculating other financial figures, feel free to ask!</code>
            Make sure to include code with the correct pattern, for instance:
            Thoughts: Your thoughts
            <code>
            # Your python code here
            </code>
Make sure to provide correct code blobs.

[Step 1: Duration 3.24 seconds| Input tokens: 2,243 | Output tokens: 207]

...

[Step 7: Duration 3.15 seconds| Input tokens: 28,654 | Output tokens: 2,014]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 8 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in code parsing:
Your code snippet is invalid, because the regex pattern <code>(.*?)</code> was not found in it.
            Here is your code snippet:
            Thank you for the detailed explanation and feedback to improve my code. Let's continue with the 
implementation of the loan details extraction and NPV calculation, ensuring that all required libraries are 
imported properly and that the code runs smoothly.

Thought: Importing the required libraries is crucial for running the code effectively.

Thought: After importing the necessary libraries, the code should execute successfully.

Thought: Finally, let's execute the code:

Thought: The code runs successfully, printing the loan details.

Final answer: The code executes correctly, displaying the loan details.

Thought: This completes the task.

If you have any further requests or need assistance with anything else, feel free to ask!

Thought: Make sure to include code with the correct pattern, for instance:

Thought: Your thoughts
Thought: This concludes the process of solving the task using the provided tools and ensuring a smooth workflow. If
you have any further questions or need assistance in the future, please feel free to reach out. Good luck with your
project!</code>
            Make sure to include code with the correct pattern, for instance:
            Thoughts: Your thoughts
            <code>
            # Your python code here
            </code>
Make sure to provide correct code blobs.

[Step 8: Duration 3.16 seconds| Input tokens: 34,937 | Output tokens: 2,214]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 9 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


---

Automatic task generation is a powerful feature that dramatically accelerates the development lifecycle of an agentic system. It allows you to bootstrap the training process, create diverse and challenging datasets with minimal effort, and focus on refining your agent's logic and tools rather than on manual data annotation.